In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Substance_Use_Disorder"
cohort = "GSE161999"

# Input paths
in_trait_dir = "../../input/GEO/Substance_Use_Disorder"
in_cohort_dir = "../../input/GEO/Substance_Use_Disorder/GSE161999"

# Output paths
out_data_file = "../../output/preprocess/Substance_Use_Disorder/GSE161999.csv"
out_gene_data_file = "../../output/preprocess/Substance_Use_Disorder/gene_data/GSE161999.csv"
out_clinical_data_file = "../../output/preprocess/Substance_Use_Disorder/clinical_data/GSE161999.csv"
json_path = "../../output/preprocess/Substance_Use_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Network preservation reveals shared and unique biological processes associated with chronic alcohol abuse in the NAc and PFC"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: prefrontal cortex'], 1: ['diagnosis: Alcohol', 'diagnosis: Control'], 2: ['age: 61', 'age: 44', 'age: 62', 'age: 56', 'age: 63', 'age: 42', 'age: 46', 'age: 52', 'age: 43', 'age: 59', 'age: 54', 'age: 39', 'age: 73', 'age: 50', 'age: 51', 'age: 64', 'age: 55', 'age: 47', 'age: 53', 'age: 82', 'age: 57'], 3: ['Sex: Male'], 4: ['rin: 3.6', 'rin: 3.7', 'rin: 3.4', 'rin: 2.1', 'rin: 5.2', 'rin: 5.8', 'rin: 1.4', 'rin: 3.8', 'rin: 2.8', 'rin: 2.9', 'rin: 2.6', 'rin: 2.5', 'rin: 7.8', 'rin: 5', 'rin: 7.2', 'rin: 7.9', 'rin: 4.3', 'rin: 6.6', 'rin: 2.2', 'rin: 8.3', 'rin: 3.1', 'rin: 7.4', 'rin: 4.4', 'rin: 8', 'rin: 3.2'], 5: ['brain weight: 1340', 'brai

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Looking at the background info, this appears to be a gene expression study of alcohol abuse
# in the prefrontal cortex, so gene data should be available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait: Substance Use Disorder - from row 1 (diagnosis: Alcohol/Control)
trait_row = 1

# Age: available in row 2
age_row = 2

# Gender: row 3 has Sex but it's only "Male" so it's a constant feature
gender_row = None  # Constant feature (all male)

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert trait value to binary (0 for Control, 1 for Alcohol)"""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "alcohol":
        return 1
    elif value.lower() == "control":
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """This function is defined but won't be used since gender is constant"""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "male":
        return 1
    elif value.lower() == "female":
        return 0
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # The trait_row is not None, so we extract clinical features
    # Using the geo_select_clinical_features function from the library
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview:
{'GSM4929487': [1.0, 61.0], 'GSM4929488': [0.0, 44.0], 'GSM4929489': [0.0, 62.0], 'GSM4929490': [1.0, 56.0], 'GSM4929491': [0.0, 63.0], 'GSM4929492': [1.0, 42.0], 'GSM4929493': [0.0, 46.0], 'GSM4929494': [0.0, 56.0], 'GSM4929495': [1.0, 52.0], 'GSM4929496': [0.0, 43.0], 'GSM4929497': [1.0, 59.0], 'GSM4929498': [1.0, 56.0], 'GSM4929499': [1.0, 54.0], 'GSM4929500': [1.0, 46.0], 'GSM4929501': [1.0, 39.0], 'GSM4929502': [1.0, 73.0], 'GSM4929503': [0.0, 56.0], 'GSM4929504': [0.0, 50.0], 'GSM4929505': [1.0, 63.0], 'GSM4929506': [1.0, 50.0], 'GSM4929507': [1.0, 50.0], 'GSM4929508': [1.0, 51.0], 'GSM4929509': [1.0, 64.0], 'GSM4929510': [1.0, 55.0], 'GSM4929511': [0.0, 55.0], 'GSM4929512': [0.0, 47.0], 'GSM4929513': [0.0, 50.0], 'GSM4929514': [0.0, 55.0], 'GSM4929515': [1.0, 53.0], 'GSM4929516': [0.0, 82.0], 'GSM4929517': [0.0, 64.0], 'GSM4929518': [1.0, 73.0], 'GSM4929519': [0.0, 73.0], 'GSM4929520': [0.0, 57.0], 'GSM4929521': [0.0, 59.0]}
Clinical data saved to ../.

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 73
Header line: "ID_REF"	"GSM4929487"	"GSM4929488"	"GSM4929489"	"GSM4929490"	"GSM4929491"	"GSM4929492"	"GSM4929493"	"GSM4929494"	"GSM4929495"	"GSM4929496"	"GSM4929497"	"GSM4929498"	"GSM4929499"	"GSM4929500"	"GSM4929501"	"GSM4929502"	"GSM4929503"	"GSM4929504"	"GSM4929505"	"GSM4929506"	"GSM4929507"	"GSM4929508"	"GSM4929509"	"GSM4929510"	"GSM4929511"	"GSM4929512"	"GSM4929513"	"GSM4929514"	"GSM4929515"	"GSM4929516"	"GSM4929517"	"GSM4929518"	"GSM4929519"	"GSM4929520"	"GSM4929521"
First data line: "hsa-let-7a-2-star_st"	1.82741	3.8846	2.3203	1.6715	2.68131	2.69626	1.81954	2.3203	2.3203	2.25006	2.07315	2.74054	2.17867	2.32175	2.09534	1.92895	2.40712	2.59145	2.40434	2.78593	2.10078	1.74555	2.78988	2.98177	2.66927	2.02203	2.39759	2.91484	2.12533	2.27246	2.22079	2.95433	2.49163	2.56559	2.57982
Index(['hsa-let-7a-2-star_st', 'hsa-let-7a-star_st', 'hsa-let-7a_st',
       'hsa-let-7b-star_st', 'hsa-let-7b_st', 'hsa-let-7c_st',
       'hsa-let-7d-star_st', 'hsa-let-7d_st', 

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the output
# The identifiers like "1007_s_at", "1053_at", etc. are probe IDs from Affymetrix microarrays
# These are not standard human gene symbols (which would look like BRCA1, TP53, etc.)
# They need to be mapped to human gene symbols for meaningful biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE161999
Line 6: !Series_title = Network preservation reveals shared and unique biological processes associated with chronic alcohol abuse in the NAc and PFC
Line 7: !Series_geo_accession = GSE161999
Line 8: !Series_status = Public on Nov 24 2020
Line 9: !Series_submission_date = Nov 23 2020
Line 10: !Series_last_update_date = Nov 29 2022
Line 11: !Series_pubmed_id = 33332381
Line 12: !Series_summary = This SuperSeries is composed of the SubSeries listed below.
Line 13: !Series_overall_design = Refer to individual Series
Line 14: !Series_type = Expression profiling by array
Line 15: !Series_type = Genome binding/occupancy profiling by high throughput sequencing
Line 16: !Series_sample_id = GSM4929029
Line 


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation dataframe contain probe IDs and gene symbols
# From examining the gene annotation preview, we can see:
# - 'ID' column contains probe IDs like "1007_s_at" that match the gene expression data
# - 'Gene Symbol' column contains the human gene symbols we need

# 2. Create a gene mapping dataframe by extracting these two columns
mapping_data = gene_annotation.loc[:, ['ID', 'Gene Symbol']]
mapping_data = mapping_data.dropna()  # Remove rows with missing values
mapping_data = mapping_data.rename(columns={'Gene Symbol': 'Gene'}).astype({'ID': 'str'})

# Preview the mapping data
print("Gene mapping preview (first 5 rows):")
print(mapping_data.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# Use the library function to handle the many-to-many relation between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Preview the gene expression data
print("\nGene expression data preview (first 5 genes, first 5 samples):")
print(gene_data.iloc[:5, :5])

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Gene mapping preview (first 5 rows):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A

Gene expression data preview (first 5 genes, first 5 samples):
Empty DataFrame
Columns: [GSM4929487, GSM4929488, GSM4929489, GSM4929490, GSM4929491]
Index: []

Gene expression data saved to ../../output/preprocess/Substance_Use_Disorder/gene_data/GSE161999.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print(f"Gene data shape before normalization: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Dataset contains gene expression data but lacks clear trait indicators for {trait} status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains gene expression data for {trait} analysis."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data for {trait}: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Gene data shape before normalization: (0, 35)
Gene data shape after normalization: (0, 35)
Normalized gene data saved to ../../output/preprocess/Substance_Use_Disorder/gene_data/GSE161999.csv
Raw clinical data shape: (13, 36)
Clinical features:
                        GSM4929487  GSM4929488  GSM4929489  GSM4929490  \
Substance_Use_Disorder         1.0         0.0         0.0         1.0   
Age                           61.0        44.0        62.0        56.0   

                        GSM4929491  GSM4929492  GSM4929493  GSM4929494  \
Substance_Use_Disorder         0.0         1.0         0.0         0.0   
Age                           63.0        42.0        46.0        56.0   

                        GSM4929495  GSM4929496  ...  GSM4929512  GSM4929513  \
Substance_Use_Disorder         1.0         0.0  ...         0.0         0.0   
Age                           52.0        43.0  ...        47.0        50.0   

                        GSM4929514  GSM4929515  GSM4929516  GSM4929517 

/media/techt/DATA/GenoAgent/tools/preprocess.py:400: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  linked_data = pd.concat([clinical_df, genetic_df], axis=0).T
